In [ ]:
%pip install transformers

In [ ]:
from tqdm import tqdm
import s3fs
import os

# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "cthiounn2"
fs.ls(BUCKET)

files=['fusion_pokemon.parquet']
for file in tqdm(files):
    with fs.open(f'{BUCKET}/{file}', mode="rb") as file_in, open(file,"wb") as file_out:
            file_out.write(file_in.read())

In [ ]:
import pandas as pd
df=pd.read_parquet("fusion_pokemon.parquet")


In [ ]:
df=df.rename(columns={"Nom français": "caption"})

In [ ]:
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
max_length=255
padding="max_length"
df["token_caption"]=df.apply(lambda x:tokenizer(x.caption, return_tensors="pt",max_length=max_length,padding=padding)['input_ids'].squeeze().numpy(), axis=1)

In [ ]:
df

In [ ]:
df[df.caption.str.contains('Pi')]

In [ ]:
df[df.caption.str.contains('zarre')]

In [ ]:
df[df.caption.str.contains('Flor')]

In [ ]:
df2=df[['token_caption','encoding']]

In [ ]:
df2

In [ ]:
df_train,df_test=df2[:150],df2[150:]
df_train

In [ ]:
df_test

In [ ]:
df_train.to_parquet("train_pokemon.parquet")
df_test.to_parquet("test_pokemon.parquet")